## 네이버 영화리뷰 감성분석 도전하기
\$ wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
\$ wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
\$ mv ratings_*.txt ~/aiffel/sentiment_classification

$ python3
```
>>> from konlpy.tag import Komoran
>>> komoran = Komoran() 
```
에서 계속 오류가 났지만,  

https://zereight.tistory.com/650  
덕분에 Komoran을 돌릴 수 있다.

https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh


In [1]:
import pandas as pd
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import Counter

# 데이터를 읽어봅시다. 
train_data = pd.read_table('~/aiffel/sentiment_classification/ratings_train.txt')
test_data = pd.read_table('~/aiffel/sentiment_classification/ratings_test.txt')

train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


2) 데이터로더 구성

실습때 다루었던 IMDB 데이터셋은 텍스트를 가공하여 imdb.data_loader() 메소드를 호출하면 숫자 인덱스로 변환된 텍스트와 word_to_index 딕셔너리까지 친절하게 제공합니다. 그러나 이번에 다루게 될 nsmc 데이터셋은 전혀 가공되지 않은 텍스트 파일로 이루어져 있습니다. 이것을 읽어서 imdb.data_loader()와 동일하게 동작하는 자신만의 data_loader를 만들어 보는 것으로 시작합니다. data_loader 안에서는 다음을 수행해야 합니다.

    데이터의 중복 제거
    NaN 결측치 제거
    한국어 토크나이저로 토큰화
    불용어(Stopwords) 제거
    사전word_to_index 구성
    텍스트 스트링을 사전 인덱스 스트링으로 변환
    X_train, y_train, X_test, y_test, word_to_index 리턴

In [2]:
# 시간오래걸린다
from konlpy.tag import Mecab
tokenizer = Mecab()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

def load_data(train_data, test_data, num_words=10000):
    train_data.drop_duplicates(subset=['document'], inplace=True)
    train_data = train_data.dropna(how = 'any') 
    test_data.drop_duplicates(subset=['document'], inplace=True) # 어떤 비율로 train과 test를 나눴는지 모르겠음
    test_data = test_data.dropna(how = 'any') 

    x_train = []
    for sentence in train_data['document']:
        temp_x = tokenizer.morphs(sentence) # 토큰화
        temp_x = [word for word in temp_x if not word in stopwords] # 불용어 제거
        x_train.append(temp_x)

    x_test = []
    for sentence in test_data['document']:
        temp_x = tokenizer.morphs(sentence) # 토큰화
        temp_x = [word for word in temp_x if not word in stopwords] # 불용어 제거
        x_test.append(temp_x)

    words = np.concatenate(x_train).tolist()
    counter = Counter(words)
    counter = counter.most_common(10000-4)
    vocab = ['<PAD>', '<BOS>', '<UNK>', '<UNUSED>'] + [key for key, _ in counter]
    word_to_index = {word:index for index, word in enumerate(vocab)} # enumerate 열거하다
#     index_to_word = {index:word for word, index in word_to_index.items()}

    def wordlist_to_indexlist(wordlist):
        return [word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in wordlist]

    x_train = list(map(wordlist_to_indexlist, x_train))
    x_test = list(map(wordlist_to_indexlist, x_test))

    return x_train, np.array(list(train_data['label'])), x_test, np.array(list(test_data['label'])), word_to_index

x_train, y_train, x_test, y_test, word_to_index = load_data(train_data, test_data)

In [3]:
print(x_train[3]) # 왜 BOS로 시작 안하지?

[8410, 158, 3925, 4, 4, 298, 94, 17, 6, 4, 4, 58, 3479]


In [4]:
index_to_word = {index:word for word, index in word_to_index.items()}

In [5]:

# 숫자 벡터로 encode된 문장을 원래대로 decode하는 함수입니다. 
def get_decoded_sentence(encoded_sentence, index_to_word):
    return ' '.join(index_to_word[index] if index in index_to_word else '<UNK>' for index in encoded_sentence[:])

# 여러개의 숫자 벡터로 encode된 문장을 한꺼번에 원래대로 decode하는 함수입니다. 
def get_decoded_sentences(encoded_sentences, index_to_word):
    return [get_decoded_sentence(encoded_sentence, index_to_word) for encoded_sentence in encoded_sentences]


print(x_train[3])
print(get_decoded_sentence(x_train[3], index_to_word))
print('라벨: ', y_train[3])  # 6번째 리뷰데이터의 라벨 부정은 0, 긍정은 1

# x_train에 아직 BOS 안 들어감

[8410, 158, 3925, 4, 4, 298, 94, 17, 6, 4, 4, 58, 3479]
교도소 이야기 구먼 . . 솔직히 재미 없 다 . . 평점 조정
라벨:  0


In [6]:
print(index_to_word[8410])
print(index_to_word[158]) 

교도소
이야기


In [7]:
print(len(x_train))
print(len(x_test))
print(len(word_to_index))

146182
49157
10000


In [8]:
# 문장 1개를 활용할 딕셔너리와 함께 주면, 단어 인덱스 리스트 벡터로 변환해 주는 함수입니다. 
# 단, 모든 문장은 <BOS>로 시작하는 것으로 합니다. 
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index['<BOS>']]+[word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in sentence.split()]

print(get_encoded_sentence('아이언맨 영화 좋다', word_to_index))

# 여러 개의 문장 리스트를 한꺼번에 단어 인덱스 리스트 벡터로 encode해 주는 함수입니다. 
def get_encoded_sentences(sentences, word_to_index):
    return [get_encoded_sentence(sentence, word_to_index) for sentence in sentences]

encoded_sentences = get_encoded_sentences('개노잼', word_to_index) # 이해안감
print(encoded_sentences)


[1, 3473, 5, 2]
[[1, 122], [1, 269], [1, 102]]


In [9]:
# 지금 보니까 보정 안하는 게 맞음
# # 보정 -> 실제 인코딩 인덱스는 제공된 word_to_index에서 index 기준으로 3씩 뒤로 밀려 있습니다.  
# word_to_index = {k:(v+3) for k,v in word_to_index.items()}

# # 처음 몇 개 인덱스는 사전에 정의되어 있습니다
# word_to_index["<PAD>"] = 0
# word_to_index["<BOS>"] = 1
# word_to_index["<UNK>"] = 2  # unknown
# word_to_index["<UNUSED>"] = 3

# index_to_word[0] = "<PAD>"
# index_to_word[1] = "<BOS>"
# index_to_word[2] = "<UNK>"
# index_to_word[3] = "<UNUSED>"

# index_to_word = {index:word for word, index in word_to_index.items()}

# print(x_train[3])
# print(get_decoded_sentence(x_train[3], index_to_word))
# print('라벨: ', y_train[3])  # 6번째 리뷰데이터의 라벨 부정은 0, 긍정은 1

In [10]:
print(index_to_word[0]) 
print(index_to_word[1]) 
print(index_to_word[2]) 
print(index_to_word[3])  # padding 확인

<PAD>
<BOS>
<UNK>
<UNUSED>


In [11]:
nn = 3
print(x_train[nn])
print(get_decoded_sentence(x_train[nn], index_to_word))
print('라벨: ', y_train[nn])  # (nn+1)번째 리뷰데이터의 라벨 부정은 0, 긍정은 1

[8410, 158, 3925, 4, 4, 298, 94, 17, 6, 4, 4, 58, 3479]
교도소 이야기 구먼 . . 솔직히 재미 없 다 . . 평점 조정
라벨:  0


In [12]:
nn = 500
print(x_train[nn])
print(get_decoded_sentence(x_train[nn], index_to_word))
print('라벨: ', y_train[nn])  # (nn+1)번째 리뷰데이터의 라벨 부정은 0, 긍정은 1

[20, 118, 10, 7, 2262, 418, 27, 900]
나 이거 보 고 인형 절대 안 삼
라벨:  1


encoding 숫자 출력할 때 왜 1이 안나올까?

In [13]:
total_data_text = list(x_train) + list(x_test)
# 텍스트 데이터 문장길이의 리스트를 생성한 후
num_tokens = [len(tokens) for tokens in total_data_text]
num_tokens = np.array(num_tokens)
# 문장길이의 평균값, 최대값, 표준편차를 계산해 본다. 
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

# 예를들어, 최대 길이를 (평균 + 2*표준편차)로 한다면,  
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)
print('pad_sequences maxlen : ', maxlen)
print('전체 문장의 {}%가 maxlen 설정값 이내에 포함됩니다. '.format(np.sum(num_tokens < max_tokens)*100 / len(num_tokens)))

문장길이 평균 :  15.969376315021577
문장길이 최대 :  116
문장길이 표준편차 :  12.843535456326455
pad_sequences maxlen :  41
전체 문장의 93.42988343341575%가 maxlen 설정값 이내에 포함됩니다. 


In [14]:
x_train = keras.preprocessing.sequence.pad_sequences(x_train,
                                                       value=word_to_index["<PAD>"],
                                                       padding='pre', # for RNN
                                                       maxlen=maxlen)

x_test = keras.preprocessing.sequence.pad_sequences(x_test,
                                                      value=word_to_index["<PAD>"],
                                                     padding='pre', # for RNN
                                                      maxlen=maxlen)

print('(x_train data 개수, 각 data의 length) =', x_train.shape)

(x_train data 개수, 각 data의 length) = (146182, 41)


In [15]:
# index_to_word = {index:word for word, index in word_to_index.items()}

In [16]:
nn = 3
print(x_train[nn])
print(get_decoded_sentence(x_train[nn], index_to_word))
print('라벨: ', y_train[nn])  # (nn+1)번째 리뷰데이터의 라벨 부정은 0, 긍정은 1

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
 8410  158 3925    4    4  298   94   17    6    4    4   58 3479]
<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 교도소 이야기 구먼 . . 솔직히 재미 없 다 . . 평점 조정
라벨:  0


In [17]:
vocab_size = 10000    # 어휘 사전의 크기입니다(10,000개의 단어)
word_vector_dim = 16  # 워드 벡터의 차원수 (변경가능한 하이퍼파라미터)

# model 설계 - RNN
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,))) # 동일 # model의 첫번째 레이어
model.add(keras.layers.Conv1D(16, 7, activation='relu'))
model.add(keras.layers.MaxPooling1D(5))
model.add(keras.layers.Conv1D(16, 7, activation='relu'))
model.add(keras.layers.GlobalMaxPooling1D())
model.add(keras.layers.Dense(8, activation='relu')) # 동일
model.add(keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다. # 동일

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160000    
_________________________________________________________________
conv1d (Conv1D)              (None, None, 16)          1808      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 16)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 16)          1808      
_________________________________________________________________
global_max_pooling1d (Global (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 8)                 136       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9

In [18]:
# validation set 분리
x_val = x_train[:30000]   
y_val = y_train[:30000]

# validation set을 제외한 나머지 15000건
partial_x_train = x_train[30000:]  
partial_y_train = y_train[30000:]

print(partial_x_train.shape)
print(partial_y_train.shape)

(116182, 41)
(116182,)


In [19]:
# model 학습 시작 "complie"
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
              
epochs=20  # 몇 epoch를 훈련하면 좋을지 결과를 보면서 바꾸어 봅시다. 

# fit에 주목
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=epochs,
                    batch_size=1500,
                    validation_data=(x_val, y_val),
                    verbose=1)

Epoch 1/20
78/78 [==============================] - 5s 61ms/step - loss: 0.6306 - accuracy: 0.6731 - val_loss: 0.4379 - val_accuracy: 0.8127
Epoch 2/20
78/78 [==============================] - 0s 6ms/step - loss: 0.3745 - accuracy: 0.8385 - val_loss: 0.3565 - val_accuracy: 0.8445
Epoch 3/20
78/78 [==============================] - 0s 5ms/step - loss: 0.3242 - accuracy: 0.8644 - val_loss: 0.3495 - val_accuracy: 0.8480
Epoch 4/20
78/78 [==============================] - 0s 5ms/step - loss: 0.3027 - accuracy: 0.8744 - val_loss: 0.3503 - val_accuracy: 0.8500
Epoch 5/20
78/78 [==============================] - 0s 5ms/step - loss: 0.2863 - accuracy: 0.8834 - val_loss: 0.3519 - val_accuracy: 0.8498
Epoch 6/20
78/78 [==============================] - 0s 5ms/step - loss: 0.2704 - accuracy: 0.8917 - val_loss: 0.3542 - val_accuracy: 0.8484
Epoch 7/20
78/78 [==============================] - 0s 5ms/step - loss: 0.2529 - accuracy: 0.8997 - val_loss: 0.3592 - val_accuracy: 0.8483
Epoch 8/20
78/78 [=

In [20]:
# test 는 "evaluate"
results = model.evaluate(x_test,  y_test, verbose=2)

print(results)

1537/1537 - 3s - loss: 0.7252 - accuracy: 0.8191
[0.7251625061035156, 0.819069504737854]


3) 모델구성을 위한 데이터 분석 및 가공

    데이터셋 내 문장 길이 분포
    적절한 최대 문장 길이 지정
    keras.preprocessing.sequence.pad_sequences 을 활용한 패딩 추가

4) 모델구성 및 validation set 구성

모델은 3가지 이상 다양하게 구성하여 실험해 보세요.
5) 모델 훈련 개시
6) Loss, Accuracy 그래프 시각화
7) 학습된 Embedding 레이어 분석
8) 한국어 Word2Vec 임베딩 활용하여 성능개선

한국어 Word2Vec은 다음 경로에서 구할 수 있습니다.

https://github.com/Kyubyong/wordvectors

평가문항	
 상세기준

1. 다양한 방법으로 Text Classification 태스크를 성공적으로 구현하였다.
	3가지 이상의 모델이 성공적으로 시도됨

2. gensim을 활용하여 자체학습된 혹은 사전학습된 임베딩 레이어를 분석하였다.
	gensim의 유사단어 찾기를 활용하여 자체학습한 임베딩과 사전학습 임베딩을 적절히 분석함

3. 한국어 Word2Vec을 활용하여 가시적인 성능향상을 달성했다.
	네이버 영화리뷰 데이터 감성분석 정확도를 85% 이상 달성함
